# Traitement et stockage des données issues du scraping

Vous avez scrapé les données du site de livres et les avez stockées dans un fichier. 

L'objectif de ce notebook est de créer une base de données pour y stocker ces données.

In [278]:
import sqlite3
import pandas as pd

Lire les données du fichier sauvegardé en utilisant pandas.

In [279]:
# Lire les données du fichier que vous venez d'enregistrer
df_books = pd.read_csv('../books_infos.csv')
df_books = df_books.rename(columns={"Unnamed: 0":"index"})
df_books

,index,title,price,rating,availability
0,0,A Light in the Attic,£51.77,Three,In stock
1,1,Tipping the Velvet,£53.74,One,In stock
2,2,Soumission,£50.10,One,In stock
3,3,Sharp Objects,£47.82,Four,In stock
4,4,Sapiens: A Brief History of Humankind,£54.23,Five,In stock
...,...,...,...,...,...
995,995,Alice in Wonderland (Alice's Adventures in Won...,£55.53,One,In stock
996,996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",£57.06,Four,In stock
997,997,A Spy's Devotion (The Regency Spies of London #1),£16.97,Five,In stock
998,998,1st to Die (Women's Murder Club #1),£53.98,One,In stock


## 1. Prétraitement des données

On souhaite créer la table _book_ contenant les attributs suivants : 
- id : INT, PK,
- title : TEXT,
- price : DECIMAL
- availability : BOOLEAN
- rating : INT [0:5]

Vérifier les types des colonnes du dataframe.

In [280]:
# Vérification des types de données
print(df_books.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         1000 non-null   int64 
 1   title         1000 non-null   object
 2   price         1000 non-null   object
 3   rating        1000 non-null   object
 4   availability  1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB
None


Dans les cellules qui suivent, des méthodes de traitement de données sont suggérées pour donner un aperçu de ce qu'il est possible de faire avec pandas.

**Il est tout à fait possible de faire autrement.**

Utiliser la méthode pandas [_astype_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) pour convertir la colonne de titre en chaîne de caractère.

In [281]:
# Conversion de title en chaîne de caractères
df_books["title"] = df_books["title"].astype("string")

# Vérification du type de la colonne title
print(df_books["title"].dtype)

string


Pour convertir la colonne de prix en nombre décimal, il est nécessaire d'utiliser une étape intermédiaire pour retirer le caractère "£".

Il est possible par exemple d'utiliser l'attribut [.str](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html) de la série "price".

In [282]:
# Convertir la colonne price en type décimal
df_books["price"] = df_books['price'].str.slice(1).astype("float")
# Vérification du type de la colonne price

print(df_books["price"].dtype)

float64


Convertir la colonne `availability` en boolen (True/False).

Quelles sont les valeurs possibles pour la colonne availability ?

In [283]:
# Valeurs possibles de la colonne availability

# Les valeurs possible pour la colonne availability sont "In stock" et "Out of stock"

Créer une fonction qui prend en entrée la valeur de `availability` et qui renvoie True ou False en fonction de la valeur d'entrée.

In [284]:
# Fonction pour convertir la valeur de availability en booléen
def convert_availability(value : str) -> bool:
    if value == "In stock":
        return True
    else:
        return False
    """Convert the availability value to a boolean.

    Args:
        value (str): The availability status of the book.

    Returns:
        bool: True if the book is available, False otherwise.
    """
    

Utiliser la méthode [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) pour appliquer la fonction à la colonne `availability`.

In [285]:
# Convertir la colonne availability en booléen (True/False)
df_books["availability"] = df_books["availability"].apply(convert_availability)

# Vérification du type de la colonne availability
print(df_books["availability"].dtype)

bool


Convertir la colonne _rating_ en chiffre en utilisant un dictionnaire `rating_map` et la méthode [_map_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html).

In [286]:
# Dictionnaire associant les notes au format initial et les valeurs numérique
ratings_map = ({"One":1,"Two":2,"Three":3,"Four":4,"Five":5})

df_books["rating"] = df_books["rating"].map(ratings_map)

# Vérification du type de la colonne rating
print(df_books["rating"].dtype)

int64


In [287]:
# Créer une fonction convert_types qui combine les traitements faits dans les cellules précédentes
def convert_types(df_books: pd.DataFrame) -> pd.DataFrame:
    df_books["title"] = df_books["title"].astype("string")
    df_books["rating"] = df_books["rating"].map({"One":1,"Two":2,"Three":3,"Four":4,"Five":5})
    df_books["price"] = df_books['price'].str.slice(1).astype("float")
    df_books["availability"] = df_books["availability"].map({"In stock":True,"Out of stock":False})
    return df_books

    """Convert the types of the DataFrame columns to appropriate types.

    Args:
        df_books (pd.DataFrame): The DataFrame containing book data.

    Returns:
        pd.DataFrame: The DataFrame with converted types.
    """

print(df_books.columns)



Index(['index', 'title', 'price', 'rating', 'availability'], dtype='object')


---
## 2. Insertion des données en base

Dans cette section :
- on créé une BDD sqlite  `book_store.db` (ou on se connecte à la base si elle existe déjà) en utilisant la bibliothèque python sqlite3,
- on insère les données prétraitées dans la BDD

Utiliser le [tutoriel](https://www.ionos.fr/digitalguide/sites-internet/developpement-web/sqlite3-avec-python/) pour l'utilisation de sqlite3.

Utiliser la fonction pandas adaptée qui permet d'insérer un dataframe dans une BDD.

In [288]:
# Création de la BDD et insertion des données
connection = sqlite3.connect("book_store.db")

df_books.to_sql("books",connection, if_exists="replace", index=False)

1000

Vérifier le nombre de livres présents dans la BDD en utilisant sqlite3 et la requête SQL adaptée.

In [289]:
# Compter le nombre de livre dans la BDD
cursor = connection.cursor()
cursor.execute("SELECT COUNT(*) FROM books")
rows = cursor.fetchall()
print(rows[0])
connection.close()

(1000,)
